In [49]:
import time

import cv2 as cv
import numpy as np
import mediapipe as mp

In [50]:
# Coordinates of eye points that will be used to calculate the EAR (Eye Aspect Ratio)
p_left_eye = [385, 380, 387, 373, 362, 263]
p_right_eye = [160, 144, 158, 153, 33, 133]
p_eyes = p_left_eye + p_right_eye

p_mouth = [82, 87, 13, 14, 312, 317, 78, 308]

In [51]:
# variables and constants used along the code
EAR_THRESHOLD = 0.3
MAR_THRESHOLD = 0.1
TIME_THRESHOLD = 2
sleeping = False
blink_count = 0


In [52]:
def calculate_ear(face, right_eye: list, left_eye: list) -> float:
    """
    Calculate the EAR (Eye Aspect Ratio) of the face.
    :param face: face mesh returned by MediaPipe
    :param right_eye: list of eye points of the right eye
    :param left_eye: list of eye points of the left eye
    :return: EAR of the face
    """
    
    face_array = np.array([[coord.x, coord.y] for coord in face])
    
    left_face = face_array[left_eye, :]
    right_face = face_array[right_eye, :]
    
    left_ear = (np.linalg.norm(left_face[0]-left_face[1]) + np.linalg.norm(left_face[2]-left_face[3])) / (2*np.linalg.norm(left_face[4]-left_face[5]))
    
    right_ear = (np.linalg.norm(right_face[0]-right_face[1]) + np.linalg.norm(right_face[2]-right_face[3])) / (2*np.linalg.norm(right_face[4]-right_face[5]))
    
    return (left_ear + right_ear) / 2

In [53]:
def calculate_mar(face, mouth: list) -> float:
    """
    Calculate the MAR (Mouth Aspect Ratio) of the face.
    :param face: face mesh returned by MediaPipe
    :param mouth: list of mouth points
    :return: MAR of the face
    """
    
    face_array = np.array([[coord.x, coord.y] for coord in face])
    
    mouth_face = face_array[mouth, :]
    
    mar = (np.linalg.norm(mouth_face[0]-mouth_face[1]) + np.linalg.norm(mouth_face[2]-mouth_face[3]) + np.linalg.norm(mouth_face[4]-mouth_face[5])) / (2*np.linalg.norm(mouth_face[6]-mouth_face[7]))
    
    return mar

In [54]:
# Defining:
# 1. The drawing utilities of MediaPipe
# 2. The FaceMesh model
# 3. The video capture
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
cap = cv.VideoCapture(0)


with mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    # refine_landmarks=True ## Este parâmetro pode ser utilizado para detectar íris
) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Ignoring empty camera frame.")
            continue

        height, width, _ = frame.shape
        
        # Flip the image horizontally for a later selfie-view display
        frame = cv.cvtColor(cv.flip(frame, 1), cv.COLOR_BGR2RGB)
        
        # Send the frame to MediaPipe FaceMesh and get the result
        results = face_mesh.process(frame)
        
        #Converting the image back to BGR
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            # Iterating over the landmarks detected
            for face_landmarks in results.multi_face_landmarks:
                # Drawing the face landmarks on the frame                
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1),
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                )
                        
                face = face_landmarks.landmark
                        
                for id_coord, coord_xyz in enumerate(face):
                    # Checking if the landmark is an eye point
                    if id_coord in p_eyes:
                        coord_cv = mp_drawing._normalized_to_pixel_coordinates(coord_xyz.x, coord_xyz.y, width, height)
                    
                        cv.circle(frame, coord_cv, 2, (255, 0, 0), -1)
                    
                    #Checking if the landmark is a mouth point
                    if id_coord in p_mouth:
                        coord_cv = mp_drawing._normalized_to_pixel_coordinates(coord_xyz.x, coord_xyz.y, width, height)
                    
                        cv.circle(frame, coord_cv, 2, (255, 0, 0), -1)
                ear = calculate_ear(face, p_right_eye, p_left_eye)
                mar = calculate_mar(face, p_mouth)
                
                # Provisionally displaying the EAR on the screen                
                cv.rectangle(frame, (0,1), (290, 140), (58, 58, 55), -1)
                cv.putText(frame, f'EAR: {round(ear, 2)}', (1, 24), cv.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
                
                # Provisionally displaying the MAR on the screen
                cv.putText(frame, f'MAR: {round(mar, 2)}', (1, 50), cv.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
                
                # Checking if the EAR is below the threshold. If so, algorithm starts counting the time
                if ear < EAR_THRESHOLD and mar < MAR_THRESHOLD:
                    t_init = time.time() if not sleeping else t_init
                    blink_count += 1 if not sleeping else blink_count
                    sleeping = True
                
                # If the EAR is above the threshold, the algorithm stops counting the time    
                if (sleeping and ear > EAR_THRESHOLD) or (ear <= EAR_THRESHOLD and mar >= MAR_THRESHOLD):
                    sleeping = False
                
                t_end = time.time()
                t = t_end - t_init if sleeping else 0
                
                cv.putText(frame, f'Blinks: {blink_count}', (1, 120), cv.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
                cv.putText(frame, f'Time: {round(t, 3)}', (1, 80), cv.FONT_HERSHEY_DUPLEX, 0.9, (255, 255, 255), 2)
                
                # If the time is above the threshold, the algorithm displays a message on the screen
                if t > TIME_THRESHOLD:
                    cv.putText(frame, 'Drowsiness Detected', (1, 120), cv.FONT_HERSHEY_DUPLEX, 0.9, (0, 0, 255), 2)
                
        cv.imshow('MediaPipe FaceMesh', frame)
        
        # Break the loop when the 'q' key is pressed
        if cv.waitKey(5) & 0xFF == ord('q'):
            break   

# Release the camera and destroy all windows
cap.release()
cv.destroyAllWindows()